In [1]:
import pandas as pd
import numpy as np
import json 
import copy
import csv
import os
import re
import nltk
import nltk
from nltk.corpus import stopwords
import sklearn
from nltk.stem import PorterStemmer
from stemming.porter2 import stem

# load our pos/neg corpus

In [2]:
pos=list(set(open("positive.txt", 'r').read().split('\',\'')))

In [3]:
neg=list(set(open("negative.txt", 'r').read().split('\',\'')))

In [4]:
pos.remove('poor')

In [5]:
for i in range(len(pos)):
    temp=str('NOT_'+str(pos[i]))
    neg.append(temp)

# data cleaning

In [6]:
data = [] 
cf = open('bars_rev_tr_df.csv','r')

file = csv.DictReader(cf)
#print(file.fieldnames)

for x in file:
    line = {'business_id':x['business_id'],'date':x['date'],'stars':x['stars'],'text':x['text']}
    data.append(line)

cf.close()

In [7]:
id = open('uniq_id_lvbars.txt','r')
id_str=id.read()
id.close()
id_list=id_str.split(',')

In [8]:
dat_night=[]
for i in range(len(data)):
    if data[i]['business_id'] in id_list:
        dat_night.append(data[i])

In [9]:
#dat_night=dat_night[0:10000]

In [9]:
reviewslist = copy.deepcopy(dat_night)
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

for ind in range(len(reviewslist)):
    texts = ''
    texts = reviewslist[ind]['text']
    texts = texts.lower()
    
    texts = re.sub('n\'t',' not', texts)
    
    texts = re.sub('isnt','isn\'t', texts)
    texts = re.sub('wasnt','wasn\'t', texts)
    texts = re.sub('werent','weren\'t', texts)
    texts = re.sub('dont','don\'t', texts)
    texts = re.sub('doesnt','doesn\'t', texts)
    texts = re.sub('didnt','didn\'t', texts)
    texts = re.sub('hasnt','hasn\'t', texts)
    texts = re.sub('havent','haven\'t', texts)
    texts = re.sub('hadnt','hadn\'t', texts)
    texts = re.sub('mightnt','mightn\'t', texts)
    texts = re.sub('shouldnt','shouldn\'t', texts)
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('weren','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)
    texts = re.sub('won\'t','will not', texts)
    
    texts = re.sub('n\'t',' not', texts)
    
    #add NOT_
    pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
        if matchObj1 != None :
            sub=re.sub(r' ', " NOT_", ' '+matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        elif matchObj2 != None :
            sub=re.sub(r' ', " NOT_", ' '+matchObj2.group(2))
            list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'never ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        else:
            sent = sent + list_text[i] + ' '
   
    text_clean=preprocess_reviews(sent)
    
    rvs2 = " ".join([stem(word) for word in text_clean.split(" ")])
    reviewsTEXT_clean[ind]['text']=rvs2
    reviewsTEXT_clean[ind]['text_split']=rvs2.split()

    
#reviewsTEXT_clean[0]

# cocktail adj for each business_id

In [84]:
#len(id_list)
cock_adj_list=[]
for i in range(len(id_list)):
    temp=[]
    count_pos=0
    count_neg=0
    list_pos=[]
    list_neg=[]
    dict={}
    dict['business_id']=id_list[i]
    for j in range(len(reviewsTEXT_clean)):
        if reviewsTEXT_clean[j]['business_id'] == id_list[i]:
            temp.append(reviewsTEXT_clean[j])
    
    word_count=0
    for m in range(len(temp)):
        word_count+=len(temp[m]['text_split'])
        if 'cocktail' in temp[m]['text_split']:
            index_list=[p for p,x in enumerate(temp[m]['text_split']) if x=='cocktail']
            for index in index_list:
                if (index-2 >= 0) and (temp[m]['text_split'][index-2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-2])
                elif (index-2 >= 0) and (temp[m]['text_split'][index-2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-2])
            
                if (index-1 >= 0) and (temp[m]['text_split'][index-1] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-1])
                elif (index-1 >= 0) and (temp[m]['text_split'][index-1] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-1])
            
                if (index+2 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+1] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+1])
                elif (index+2 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+1] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+1])
            
                if (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+2])
                elif (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+2])
            
    dict['pos_words']=list_pos
    dict['neg_words']=list_neg
    dict['pos_count']=count_pos
    dict['neg_count']=count_neg
    dict['all_mention']=count_pos+count_neg
    dict['review_count']=len(temp)
    dict['word_count']=word_count
    cock_adj_list.append(dict)

In [85]:
cock_adj_df=pd.DataFrame.from_dict(cock_adj_list)
cock_adj_df=cock_adj_df[['business_id','neg_words','neg_count','pos_words','pos_count','all_mention','review_count','word_count']]
cock_adj_df=cock_adj_df.sort_values(by="all_mention", ascending=False)
cock_adj_df.to_csv('cocktail_adj.csv',index=False)
stars = pd.read_csv('LVbars_Business_withStars.csv')
adj = pd.read_csv('cocktail_adj.csv')
df = pd.merge(adj, stars, how='left', on='business_id')

In [86]:
neg_top5=[]
for i in range(len(df)):
    dict_neg=nltk.FreqDist(df['neg_words'][i][2:(len(df['neg_words'][i])-2)].split('\', \''))
    if len(list(dict_neg.keys()))>=5:
        neg_top5.append(list(dict_neg.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        neg_top5.append(list(dict_neg.keys()))
    else:
        neg_top5.append('NULL')
df['neg_top5']=neg_top5

In [87]:
pos_top5=[]
for i in range(len(df)):
    dict_pos=nltk.FreqDist(df['pos_words'][i][2:(len(df['pos_words'][i])-2)].split('\', \''))
    if len(list(dict_pos.keys()))>=5:
        pos_top5.append(list(dict_pos.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        pos_top5.append(list(dict_pos.keys()))
    else:
        pos_top5.append('NULL')
df['pos_top5']=pos_top5

In [88]:
df=df[['business_id','neg_words','neg_count','neg_top5','pos_words','pos_count','pos_top5','all_mention','review_count','word_count','stars']]
df.to_csv('cocktail_adj.csv',index=False)

# beer adj for each business_id

In [115]:
beer_adj_list=[]
for i in range(len(id_list)):
    temp=[]
    count_pos=0
    count_neg=0
    list_pos=[]
    list_neg=[]
    dict={}
    dict['business_id']=id_list[i]
    for j in range(len(reviewsTEXT_clean)):
        if reviewsTEXT_clean[j]['business_id'] == id_list[i]:
            temp.append(reviewsTEXT_clean[j])
    
    word_count=0
    for m in range(len(temp)):
        word_count+=len(temp[m]['text_split'])
        if 'beer' in temp[m]['text_split']:
            index_list=[p for p,x in enumerate(temp[m]['text_split']) if x=='beer']
            for index in index_list:
                if (index-2 >= 0) and (temp[m]['text_split'][index-2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-2])
                elif (index-2 >= 0) and (temp[m]['text_split'][index-2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-2])
            
                if (index-1 >= 0) and (temp[m]['text_split'][index-1] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-1])
                elif (index-1 >= 0) and (temp[m]['text_split'][index-1] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-1])
            
                if (index+2 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+1] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+1])
                elif (index+2 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+1] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+1])
            
                if (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+2])
                elif (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+2])
            
    dict['pos_words']=list_pos
    dict['neg_words']=list_neg
    dict['pos_count']=count_pos
    dict['neg_count']=count_neg
    dict['all_mention']=count_pos+count_neg
    dict['review_count']=len(temp)
    dict['word_count']=word_count
    beer_adj_list.append(dict)

In [116]:
beer_adj_df=pd.DataFrame.from_dict(beer_adj_list)
beer_adj_df=beer_adj_df[['business_id','neg_words','neg_count','pos_words','pos_count','all_mention','review_count','word_count']]
beer_adj_df=beer_adj_df.sort_values(by="all_mention", ascending=False)
beer_adj_df.to_csv('cocktail_adj.csv',index=False)
stars = pd.read_csv('LVbars_Business_withStars.csv')
adj = pd.read_csv('cocktail_adj.csv')
df = pd.merge(adj, stars, how='left', on='business_id')

In [117]:
neg_top5=[]
for i in range(len(df)):
    dict_neg=nltk.FreqDist(df['neg_words'][i][2:(len(df['neg_words'][i])-2)].split('\', \''))
    if len(list(dict_neg.keys()))>=5:
        neg_top5.append(list(dict_neg.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        neg_top5.append(list(dict_neg.keys()))
    else:
        neg_top5.append('NULL')
df['neg_top5']=neg_top5

In [118]:
pos_top5=[]
for i in range(len(df)):
    dict_pos=nltk.FreqDist(df['pos_words'][i][2:(len(df['pos_words'][i])-2)].split('\', \''))
    if len(list(dict_pos.keys()))>=5:
        pos_top5.append(list(dict_pos.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        pos_top5.append(list(dict_pos.keys()))
    else:
        pos_top5.append('NULL')
df['pos_top5']=pos_top5

In [119]:
df=df[['business_id','neg_words','neg_count','neg_top5','pos_words','pos_count','pos_top5','all_mention','review_count','word_count','stars']]
df.to_csv('beer_adj.csv',index=False)

# craft beer adj for each business_id

In [99]:
#len(id_list)
craft_adj_list=[]
for i in range(len(id_list)):
    temp=[]
    count_pos=0
    count_neg=0
    list_pos=[]
    list_neg=[]
    dict={}
    dict['business_id']=id_list[i]
    for j in range(len(reviewsTEXT_clean)):
        if reviewsTEXT_clean[j]['business_id'] == id_list[i]:
            temp.append(reviewsTEXT_clean[j])
    
    word_count=0
    for m in range(len(temp)):
        word_count+=len(temp[m]['text_split'])
        if 'craft beer' in temp[m]['text']:
            index_list=[p for p,x in enumerate(temp[m]['text_split']) if x=='craft']
            for index in index_list:
                if (index-2 >= 0) and (temp[m]['text_split'][index-2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-2])
                elif (index-2 >= 0) and (temp[m]['text_split'][index-2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-2])
            
                if (index-1 >= 0) and (temp[m]['text_split'][index-1] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-1])
                elif (index-1 >= 0) and (temp[m]['text_split'][index-1] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-1])
            
                if (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+2])
                elif (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+2])
            
                if (index+4 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+3] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+3])
                elif (index+4 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+3] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+3])
            
    dict['pos_words']=list_pos
    dict['neg_words']=list_neg
    dict['pos_count']=count_pos
    dict['neg_count']=count_neg
    dict['all_mention']=count_pos+count_neg
    dict['review_count']=len(temp)
    dict['word_count']=word_count
    craft_adj_list.append(dict)

In [100]:
craft_adj_df=pd.DataFrame.from_dict(craft_adj_list)
craft_adj_df=craft_adj_df[['business_id','neg_words','neg_count','pos_words','pos_count','all_mention','review_count','word_count']]
craft_adj_df=craft_adj_df.sort_values(by="all_mention", ascending=False)
craft_adj_df.to_csv('craft_adj.csv',index=False)

adj = pd.read_csv('craft_adj.csv')
df = pd.merge(adj, stars, how='left', on='business_id')


In [101]:
neg_top5=[]
for i in range(len(df)):
    dict_neg=nltk.FreqDist(df['neg_words'][i][2:(len(df['neg_words'][i])-2)].split('\', \''))
    if len(list(dict_neg.keys()))>=5:
        neg_top5.append(list(dict_neg.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        neg_top5.append(list(dict_neg.keys()))
    else:
        neg_top5.append('NULL')
df['neg_top5']=neg_top5

In [102]:
pos_top5=[]
for i in range(len(df)):
    dict_pos=nltk.FreqDist(df['pos_words'][i][2:(len(df['pos_words'][i])-2)].split('\', \''))
    if len(list(dict_pos.keys()))>=5:
        pos_top5.append(list(dict_pos.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        pos_top5.append(list(dict_pos.keys()))
    else:
        pos_top5.append('NULL')
df['pos_top5']=pos_top5

In [103]:
df=df[['business_id','neg_words','neg_count','neg_top5','pos_words','pos_count','pos_top5','all_mention','review_count','word_count','stars']]
df.to_csv('craft_adj.csv',index=False)

# beer selection adj for each business_id

In [104]:
#len(id_list)
select_adj_list=[]
for i in range(len(id_list)):
    temp=[]
    count_pos=0
    count_neg=0
    list_pos=[]
    list_neg=[]
    dict={}
    dict['business_id']=id_list[i]
    for j in range(len(reviewsTEXT_clean)):
        if reviewsTEXT_clean[j]['business_id'] == id_list[i]:
            temp.append(reviewsTEXT_clean[j])
    
    word_count=0
    for m in range(len(temp)):
        word_count+=len(temp[m]['text_split'])
        if 'beer select' in temp[m]['text']:
            index_list=[p for p,x in enumerate(temp[m]['text_split']) if x=='select']
            for index in index_list:
                if (index-3 >= 0) and (temp[m]['text_split'][index-3] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-3])
                elif (index-3 >= 0) and (temp[m]['text_split'][index-3] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-3])
            
                if (index-2 >= 0) and (temp[m]['text_split'][index-2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-2])
                elif (index-2 >= 0) and (temp[m]['text_split'][index-2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-2])
            
                if (index+2 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+1] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+1])
                elif (index+2 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+1] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+1])
            
                if (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+2])
                elif (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+2])
            
    dict['pos_words']=list_pos
    dict['neg_words']=list_neg
    dict['pos_count']=count_pos
    dict['neg_count']=count_neg
    dict['all_mention']=count_pos+count_neg
    dict['review_count']=len(temp)
    dict['word_count']=word_count
    select_adj_list.append(dict)

In [105]:
select_adj_df=pd.DataFrame.from_dict(select_adj_list)
select_adj_df=select_adj_df[['business_id','neg_words','neg_count','pos_words','pos_count','all_mention','review_count','word_count']]
select_adj_df=select_adj_df.sort_values(by="all_mention", ascending=False)
select_adj_df.to_csv('select_adj.csv',index=False)

adj = pd.read_csv('select_adj.csv')
df = pd.merge(adj, stars, how='left', on='business_id')

In [106]:
neg_top5=[]
for i in range(len(df)):
    dict_neg=nltk.FreqDist(df['neg_words'][i][2:(len(df['neg_words'][i])-2)].split('\', \''))
    if len(list(dict_neg.keys()))>=5:
        neg_top5.append(list(dict_neg.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        neg_top5.append(list(dict_neg.keys()))
    else:
        neg_top5.append('NULL')
df['neg_top5']=neg_top5

In [107]:
pos_top5=[]
for i in range(len(df)):
    dict_pos=nltk.FreqDist(df['pos_words'][i][2:(len(df['pos_words'][i])-2)].split('\', \''))
    if len(list(dict_pos.keys()))>=5:
        pos_top5.append(list(dict_pos.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        pos_top5.append(list(dict_pos.keys()))
    else:
        pos_top5.append('NULL')
df['pos_top5']=pos_top5

In [108]:
df=df[['business_id','neg_words','neg_count','neg_top5','pos_words','pos_count','pos_top5','all_mention','review_count','word_count','stars']]
df.to_csv('select_adj.csv',index=False)

# draft beer adj for each business_id

In [110]:
draft_adj_list=[]
for i in range(len(id_list)):
    temp=[]
    count_pos=0
    count_neg=0
    list_pos=[]
    list_neg=[]
    dict={}
    dict['business_id']=id_list[i]
    for j in range(len(reviewsTEXT_clean)):
        if reviewsTEXT_clean[j]['business_id'] == id_list[i]:
            temp.append(reviewsTEXT_clean[j])
    
    word_count=0
    for m in range(len(temp)):
        word_count+=len(temp[m]['text_split'])
        if 'draft beer' in temp[m]['text']:
            index_list=[p for p,x in enumerate(temp[m]['text_split']) if x=='draft']
            for index in index_list:
                if (index-2 >= 0) and (temp[m]['text_split'][index-2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-2])
                elif (index-2 >= 0) and (temp[m]['text_split'][index-2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-2])
            
                if (index-1 >= 0) and (temp[m]['text_split'][index-1] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index-1])
                elif (index-1 >= 0) and (temp[m]['text_split'][index-1] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index-1])
            
                if (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+2])
                elif (index+3 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+2] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+2])
            
                if (index+4 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+3] in pos):
                    count_pos+=1
                    list_pos.append(temp[m]['text_split'][index+3])
                elif (index+4 <= len(temp[m]['text_split'])) and (temp[m]['text_split'][index+3] in neg):
                    count_neg+=1
                    list_neg.append(temp[m]['text_split'][index+3])
            
    dict['pos_words']=list_pos
    dict['neg_words']=list_neg
    dict['pos_count']=count_pos
    dict['neg_count']=count_neg
    dict['all_mention']=count_pos+count_neg
    dict['review_count']=len(temp)
    dict['word_count']=word_count
    draft_adj_list.append(dict)

In [111]:
draft_adj_df=pd.DataFrame.from_dict(draft_adj_list)
draft_adj_df=draft_adj_df[['business_id','neg_words','neg_count','pos_words','pos_count','all_mention','review_count','word_count']]
draft_adj_df=draft_adj_df.sort_values(by="all_mention", ascending=False)
draft_adj_df.to_csv('draft_adj.csv',index=False)

adj = pd.read_csv('draft_adj.csv')
df = pd.merge(adj, stars, how='left', on='business_id')

In [112]:
neg_top5=[]
for i in range(len(df)):
    dict_neg=nltk.FreqDist(df['neg_words'][i][2:(len(df['neg_words'][i])-2)].split('\', \''))
    if len(list(dict_neg.keys()))>=5:
        neg_top5.append(list(dict_neg.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        neg_top5.append(list(dict_neg.keys()))
    else:
        neg_top5.append('NULL')
df['neg_top5']=neg_top5

In [113]:
pos_top5=[]
for i in range(len(df)):
    dict_pos=nltk.FreqDist(df['pos_words'][i][2:(len(df['pos_words'][i])-2)].split('\', \''))
    if len(list(dict_pos.keys()))>=5:
        pos_top5.append(list(dict_pos.keys())[0:5])
    elif len(list(dict_neg.keys()))>0:
        pos_top5.append(list(dict_pos.keys()))
    else:
        pos_top5.append('NULL')
df['pos_top5']=pos_top5

In [114]:
df=df[['business_id','neg_words','neg_count','neg_top5','pos_words','pos_count','pos_top5','all_mention','review_count','word_count','stars']]
df.to_csv('draft_adj.csv',index=False)